In [1]:
from sklearn.linear_model import LinearRegression 
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.metrics import mean_squared_error
from TimewiseKfold import *
from Metrics import *

In [2]:
folds, PROVINCES_NUM = splitting_data(region=2)

In [3]:
tscv = TimeSeriesSplit(n_splits=2, test_size=PROVINCES_NUM)

def optimally_selected_features(X_train, X_validation, y_train, y_validation):
    
    selected_features = []
    mse = 9999999
    
    for i in range(1, 75):
        sfs = SequentialFeatureSelector(LinearRegression(), k_features=i, forward=True, scoring='neg_mean_squared_error', cv=tscv)
        
        features = list(sfs.fit(X_train, y_train).k_feature_names_)
        
        X_train_tmp = X_train[features]
        X_validation_tmp = X_validation[features]
        
        linear_model = LinearRegression().fit(X_train_tmp, y_train)
        
        mse_tmp = mean_squared_error(y_true=y_validation, y_pred=linear_model.predict(X_validation_tmp))
        
        if (mse_tmp < mse):
            mse = mse_tmp 
            selected_features = features
    
    return selected_features
    

In [ ]:
table = metrics_table()

for (i, fold) in enumerate(folds):
    X, X_test, y, y_test = split_train_test(fold=fold)
    X_train, X_validation, y_train, y_validation = split_train_validation(X=X, y=y, PROVINCES_NUM=PROVINCES_NUM)
    
    selected_features = optimally_selected_features(X_train, X_validation, y_train, y_validation)
    
    print(selected_features)
    
    X = X[selected_features]
    X_test = X_test[selected_features]
    
    linear_model = LinearRegression().fit(X, y)
    
    row = [MAXIMUM_YEAR - FOLD_NUM + 1 + i]
    
    row += metrics_calculation(y_actual=y_test, y_pred=linear_model.predict(X_test), y_train=y)
    
    table.loc[len(table)] = row
    

table  

In [8]:
table

,Year,RMSE,R2_Score,MSE,MAE,RAE,RSE
0,2018.0,1.712589,0.006920,2.932961,1.555816,1.085250,1.080461
1,2019.0,3.443764,-6.457442,11.859513,2.611549,2.528143,7.496864
2,2020.0,5.736892,0.083135,32.911927,3.566421,0.960178,1.285276
